In [122]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import geopandas as gpd
from shapely.geometry import shape


inkar = pd.read_csv("../data/inkar_2022.csv",sep=";",decimal=",",dtype={'Kennziffer':str,'Kennziffer_EU':str,'Name':str}, low_memory=False)
inkar.head()

,Bereich,ID,Indikator,Raumbezug,Kennziffer,Name,Zeitbezug,Wert
0,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1995,10494.0
1,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1996,11921.0
2,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1997,13213.0
3,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1998,13301.0
4,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1999,12048.0


Anzahl von Raumbezügen prüfen

In [109]:
raumbezug_selection = ["Gemeinden"]
temp = inkar[inkar['Raumbezug'].isin(raumbezug_selection)]
temp

,Bereich,ID,Indikator,Raumbezug,Kennziffer,Name,Zeitbezug,Wert
20263528,Zentrale Orte Monitoring,151010,Zentralörtlicher Status,Gemeinden,01001000,"Flensburg, Stadt",2020,1.0
20263529,Zentrale Orte Monitoring,151010,Zentralörtlicher Status,Gemeinden,01002000,"Kiel, Landeshauptstadt",2020,1.0
20263530,Zentrale Orte Monitoring,151010,Zentralörtlicher Status,Gemeinden,01003000,"Lübeck, Hansestadt",2020,1.0
20263531,Zentrale Orte Monitoring,151010,Zentralörtlicher Status,Gemeinden,01004000,"Neumünster, Stadt",2020,1.0
20263532,Zentrale Orte Monitoring,151010,Zentralörtlicher Status,Gemeinden,01051001,Albersdorf,2020,1.0
...,...,...,...,...,...,...,...,...
25312773,Zentrale Orte Monitoring,156270,Kinositzplätze,Gemeinden,16077044,Starkenberg,2017,0.0
25312774,Zentrale Orte Monitoring,156270,Kinositzplätze,Gemeinden,16077047,Thonhausen,2017,0.0
25312775,Zentrale Orte Monitoring,156270,Kinositzplätze,Gemeinden,16077048,Treben,2017,0.0
25312776,Zentrale Orte Monitoring,156270,Kinositzplätze,Gemeinden,16077049,Vollmershain,2017,0.0


In [110]:
anzahl_unique_name = temp['Name'].nunique()
anzahl_zeilen = len(temp)
print(f"Anzahl der Zeilen: {anzahl_zeilen}")
print(f"Anzahl der Raumbezüge: {anzahl_unique_name}")
temp['Name'].unique()


Anzahl der Zeilen: 2561958
Anzahl der Raumbezüge: 10614


array(['Flensburg, Stadt', 'Kiel, Landeshauptstadt', 'Lübeck, Hansestadt',
       ..., 'Treben', 'Vollmershain', 'Windischleuba'], dtype=object)

Area von Raumbezügen überprüfen

In [90]:
data = pd.read_csv("out/csvs/indicator_Bevölkerunggesamt.csv")
data.head(10)

,Kennziffer,Wert,Zeitbezug
0,111,81817498.0,1995
1,111,82012160.0,1996
2,111,82057380.0,1997
3,111,82037011.0,1998
4,111,82163476.0,1999
5,111,82259531.0,2000
6,111,82440309.0,2001
7,111,82536680.0,2002
8,111,82531670.0,2003
9,111,82500850.0,2004


In [138]:
municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_krs",engine="pyogrio")
municipal_boundaries['AGS_int'] = municipal_boundaries.AGS_0.astype(int)
municipal_boundaries = pd.merge(data, municipal_boundaries,
                               left_on = "Kennziffer", 
                               right_on = "AGS_int")
municipal_boundaries = municipal_boundaries[municipal_boundaries["Zeitbezug"] == 2020]

def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries['area'] = municipal_boundaries['geometry'].apply(calculate_area)

min_bl = round(municipal_boundaries["area"].min(),3)
max_bl = round(municipal_boundaries["area"].max(), 3)
mean_bl = round(municipal_boundaries["area"].mean(), 3)
median_bl = round(municipal_boundaries["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))

Min: 0.1, Max: 5500.53, Mean: 831.899, Median: 740.985


In [139]:
municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_gem",engine="pyogrio")
municipal_boundaries['AGS_int'] = municipal_boundaries.AGS_0.astype(int)
municipal_boundaries = pd.merge(data, municipal_boundaries,
                               left_on = "Kennziffer", 
                               right_on = "AGS_int")
municipal_boundaries = municipal_boundaries[municipal_boundaries["Zeitbezug"] == 2020]

def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries['area'] = municipal_boundaries['geometry'].apply(calculate_area)

min_bl = round(municipal_boundaries["area"].min(),3)
max_bl = round(municipal_boundaries["area"].max(), 3)
mean_bl = round(municipal_boundaries["area"].mean(), 3)
median_bl = round(municipal_boundaries["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))

Min: 0.46, Max: 893.03, Mean: 138.131, Median: 108.89


In [140]:
municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_lan",engine="pyogrio")
municipal_boundaries['AGS_int'] = municipal_boundaries.AGS_0.astype(int)
municipal_boundaries = pd.merge(data, municipal_boundaries,
                               left_on = "Kennziffer", 
                               right_on = "AGS_int")
municipal_boundaries = municipal_boundaries[municipal_boundaries["Zeitbezug"] == 2020]

def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries['area'] = municipal_boundaries['geometry'].apply(calculate_area)

min_bl = round(municipal_boundaries["area"].min(),3)
max_bl = round(municipal_boundaries["area"].max(), 3)
mean_bl = round(municipal_boundaries["area"].mean(), 3)
median_bl = round(municipal_boundaries["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))

Min: 9.93, Max: 893.03, Mean: 468.068, Median: 484.655


In [147]:
municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_rbz",engine="pyogrio")
municipal_boundaries['SDV_ARS_int'] = municipal_boundaries.SDV_ARS.astype('int64')
def konvertiere_kennziffer(kennziffer):
    return int(str(int(kennziffer))[:8])

municipal_boundaries['SDV_ARS_int'] = municipal_boundaries['SDV_ARS_int'].apply(konvertiere_kennziffer)
municipal_boundaries = pd.merge(data, municipal_boundaries,left_on = "Kennziffer", right_on = "SDV_ARS_int")
municipal_boundaries = municipal_boundaries[municipal_boundaries["Zeitbezug"] == 2020]

def calculate_area(multipolygon):
    poly = shape(multipolygon)
    area_m2 = poly.area
    area_km2 = area_m2 / 1_000_000
    return round(area_km2, 2)

municipal_boundaries['area'] = municipal_boundaries['geometry'].apply(calculate_area)

min_bl = round(municipal_boundaries["area"].min(),3)
max_bl = round(municipal_boundaries["area"].max(), 3)
mean_bl = round(municipal_boundaries["area"].mean(), 3)
median_bl = round(municipal_boundaries["area"].median(), 3)

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))

Min: 0.31, Max: 17535.25, Mean: 8073.471, Median: 7723.12


In [118]:
#vg250_sta Flächen Ebene Staat
#vg250_lan Flächen Ebene Land
#vg250_rbz Flächen Ebene Regierungsbezirk
#vg250_krs Flächen Ebene Kreis
#vg250_gem Flächen Ebene Gemeinde
municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_krs",engine="pyogrio")
municipal_boundaries['AGS_int'] = municipal_boundaries.AGS_0.astype(int)
municipal_boundaries = pd.merge(data, municipal_boundaries,
                               left_on = "Kennziffer", 
                               right_on = "AGS_int")
municipal_boundaries = municipal_boundaries[municipal_boundaries["Zeitbezug"] == 2020]

min_bl = municipal_boundaries["Wert"].min()
max_bl = municipal_boundaries["Wert"].max()
mean_bl = municipal_boundaries["Wert"].mean()
median_bl =municipal_boundaries["Wert"].median()

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries

Min: 34001.0, Max: 3664088.0, Mean: 210557.82790697674, Median: 157975.0


,Kennziffer,Wert,Zeitbezug,OBJID,BEGINN,ADE,GF,BSG,ARS,AGS,...,SN_V2,SN_G,FK_S3,NUTS,ARS_0,AGS_0,WSK,DLM_ID,geometry,AGS_int
50,1001000,89934.0,2020,DEBKGVG200000C75,2022-12-20,4,4,1,01001,01001,...,00,000,R,DEF01,010010000000,01001000,2008-01-01,DEBKGDL20000002R,"MULTIPOLYGON (((526513.753 6075133.412, 526547...",1001000
51,1001000,89934.0,2020,DEBKGVG200000CI9,2022-12-20,4,2,1,01001,01001,...,00,000,R,DEF01,010010000000,01001000,2008-01-01,DEBKGDL20000E5M9,"MULTIPOLYGON (((531016.578 6076768.422, 531473...",1001000
102,1002000,246601.0,2020,DEBKGVG200000C76,2022-12-20,4,4,1,01002,01002,...,00,000,R,DEF02,010020000000,01002000,2006-01-01,DEBKGDL20000E43X,"MULTIPOLYGON (((575841.569 6032148.032, 575869...",1002000
103,1002000,246601.0,2020,DEBKGVG200000CIA,2022-12-20,4,2,1,01002,01002,...,00,000,R,DEF02,010020000000,01002000,2006-01-01,DEBKGDL20000001T,"MULTIPOLYGON (((575848.336 6032304.652, 575965...",1002000
129,1003000,215846.0,2020,DEBKGVG200000C77,2022-12-20,4,4,1,01003,01003,...,00,000,R,DEF03,010030000000,01003000,2006-02-01,DEBKGDL20000DYM9,"MULTIPOLYGON (((623056.151 5983746.445, 623191...",1003000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11075,16073000,102139.0,2020,DEBKGVG200000CI4,2022-12-20,4,4,1,16073,16073,...,00,000,R,DEG0I,160730000000,16073000,2019-01-01,DEBKGDL20000E6O4,"MULTIPOLYGON (((659352.786 5633725.523, 659501...",16073000
11101,16074000,82816.0,2020,DEBKGVG200000CI5,2022-12-20,4,4,1,16074,16074,...,00,000,R,DEG0J,160740000000,16074000,2009-01-01,DEBKGDL20000DZT7,"MULTIPOLYGON (((688961.348 5662575.516, 688976...",16074000
11127,16075000,79632.0,2020,DEBKGVG200000CI6,2022-12-20,4,4,1,16075,16075,...,00,000,R,DEG0K,160750000000,16075000,2009-01-01,DEBKGDL20000E0F6,"MULTIPOLYGON (((699461.667 5630277.321, 699499...",16075000
11153,16076000,96668.0,2020,DEBKGVG200000CI7,2022-12-20,4,4,1,16076,16076,...,00,000,R,DEG0L,160760000000,16076000,2009-01-01,DEBKGDL20000E65V,"MULTIPOLYGON (((724109.327 5650625.772, 724321...",16076000


In [119]:
municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_gem",engine="pyogrio")
municipal_boundaries['AGS_int'] = municipal_boundaries.AGS_0.astype(int)
municipal_boundaries = pd.merge(data, municipal_boundaries,
                               left_on = "Kennziffer", 
                               right_on = "AGS_int")
municipal_boundaries = municipal_boundaries[municipal_boundaries["Zeitbezug"] == 2020]
min_bl = municipal_boundaries["Wert"].min()
max_bl = municipal_boundaries["Wert"].max()
mean_bl = municipal_boundaries["Wert"].mean()
median_bl =municipal_boundaries["Wert"].median()
print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))
municipal_boundaries

Min: 34001.0, Max: 3664088.0, Mean: 263660.7192982456, Median: 129557.5


,Kennziffer,Wert,Zeitbezug,OBJID,BEGINN,ADE,GF,BSG,ARS,AGS,...,SN_V2,SN_G,FK_S3,NUTS,ARS_0,AGS_0,WSK,DLM_ID,geometry,AGS_int
50,1001000,89934.0,2020,DEBKGVG200000008,2022-12-20,6,4,1,010010000000,01001000,...,00,000,R,DEF01,010010000000,01001000,2008-01-01,DEBKGDL20000E5MA,"MULTIPOLYGON (((526513.753 6075133.412, 526547...",1001000
51,1001000,89934.0,2020,DEBKGVG2000008HF,2022-12-20,6,2,1,010010000000,01001000,...,00,000,R,DEF01,010010000000,01001000,2008-01-01,DEBKGDL20000003H,"MULTIPOLYGON (((531016.578 6076768.422, 531473...",1001000
102,1002000,246601.0,2020,DEBKGVG200000009,2022-12-20,6,4,1,010020000000,01002000,...,00,000,R,DEF02,010020000000,01002000,2006-01-01,DEBKGDL20000004J,"MULTIPOLYGON (((575841.569 6032148.032, 575869...",1002000
103,1002000,246601.0,2020,DEBKGVG2000008HG,2022-12-20,6,2,1,010020000000,01002000,...,00,000,R,DEF02,010020000000,01002000,2006-01-01,DEBKGDL20000E6XX,"MULTIPOLYGON (((575848.336 6032304.652, 575965...",1002000
129,1003000,215846.0,2020,DEBKGVG20000000A,2022-12-20,6,4,1,010030000000,01003000,...,00,000,R,DEF03,010030000000,01003000,2006-02-01,DEBKGDL20000DYMA,"MULTIPOLYGON (((623056.151 5983746.445, 623191...",1003000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2859,16051000,213692.0,2020,DEBKGVG2000007ZW,2022-12-20,6,4,1,160510000000,16051000,...,00,000,R,DEG01,160510000000,16051000,1994-10-12,DEBKGDL20000DXLP,"MULTIPOLYGON (((643368.580 5660697.134, 643363...",16051000
2885,16052000,92126.0,2020,DEBKGVG2000007ZX,2022-12-20,6,4,1,160520000000,16052000,...,00,000,R,DEG02,160520000000,16052000,1994-07-01,DEBKGDL20000DW2U,"MULTIPOLYGON (((717181.863 5651591.598, 717573...",16052000
2911,16053000,110731.0,2020,DEBKGVG2000007ZY,2022-12-20,6,4,1,160530000000,16053000,...,00,000,R,DEG03,160530000000,16053000,1994-07-01,DEBKGDL20000DWHN,"MULTIPOLYGON (((679577.691 5649953.262, 679657...",16053000
2937,16054000,36395.0,2020,DEBKGVG2000007ZZ,2022-12-20,6,4,1,160540000000,16054000,...,00,000,R,DEG04,160540000000,16054000,2019-01-01,DEBKGDL20000E052,"MULTIPOLYGON (((626564.155 5616707.029, 626689...",16054000


In [91]:
municipal_boundaries = gpd.read_file(filename='../data/DE_VG250.gpkg', layer="vg250_rbz",engine="pyogrio")
municipal_boundaries['SDV_ARS_int'] = municipal_boundaries.SDV_ARS.astype('int64')
def konvertiere_kennziffer(kennziffer):
    return int(str(int(kennziffer))[:8])

municipal_boundaries['SDV_ARS_int'] = municipal_boundaries['SDV_ARS_int'].apply(konvertiere_kennziffer)
municipal_boundaries = pd.merge(data, municipal_boundaries,left_on = "Kennziffer", right_on = "SDV_ARS_int")
municipal_boundaries = municipal_boundaries[municipal_boundaries["Zeitbezug"] == 2020]

min_bl = municipal_boundaries["Wert"].min()
max_bl = municipal_boundaries["Wert"].max()
mean_bl = municipal_boundaries["Wert"].mean()
median_bl =municipal_boundaries["Wert"].median()

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))

municipal_boundaries

Min: 41681.0, Max: 1488202.0, Mean: 318105.45, Median: 180111.0


,Kennziffer,Wert,Zeitbezug,OBJID,BEGINN,ADE,GF,BSG,ARS,AGS,...,SN_V2,SN_G,FK_S3,NUTS,ARS_0,AGS_0,WSK,DLM_ID,geometry,SDV_ARS_int
25,51110000,620523.0,2020,DEBKGVG200000CJ3,2022-12-20,3,4,1,051,051,...,00,000,R,DEA1,051000000000,05100000,1975-01-01,DEBKGDL20000E3WA,"MULTIPOLYGON (((304587.199 5754146.922, 304780...",51110000
51,53150000,1083498.0,2020,DEBKGVG200000CJ4,2022-12-20,3,4,1,053,053,...,00,000,R,DEA2,053000000000,05300000,1975-01-01,DEBKGDL20000E3WC,"MULTIPOLYGON (((301890.303 5600399.203, 301901...",53150000
77,55150000,316403.0,2020,DEBKGVG200000CJ5,2022-12-20,3,4,1,055,055,...,00,000,R,DEA3,055000000000,05500000,1998-04-01,DEBKGDL20000E44O,"MULTIPOLYGON (((405519.702 5814668.586, 405626...",55150000
103,57660020,74097.0,2020,DEBKGVG200000CJ6,2022-12-20,3,4,1,057,057,...,00,000,R,DEA4,057000000000,05700000,2009-01-01,DEBKGDL20000E65Q,"MULTIPOLYGON (((477607.280 5818768.839, 477707...",57660020
129,59580004,73487.0,2020,DEBKGVG200000CJ7,2022-12-20,3,4,1,059,059,...,00,000,R,DEA5,059000000000,05900000,2009-11-01,DEBKGDL20000E44I,"MULTIPOLYGON (((394543.626 5732696.180, 394595...",59580004
155,64110000,159174.0,2020,DEBKGVG200000CJ8,2022-12-20,3,4,1,064,064,...,00,000,R,DE71,064000000000,06400000,2011-07-01,DEBKGDL20000E3LX,"MULTIPOLYGON (((476929.894 5496705.669, 476984...",64110000
181,65310005,90131.0,2020,DEBKGVG200000CJ9,2022-12-20,3,4,1,065,065,...,00,000,R,DE72,065000000000,06500000,2009-01-01,DEBKGDL20000DZXT,"MULTIPOLYGON (((477948.925 5648954.563, 478206...",65310005
207,66110000,201048.0,2020,DEBKGVG200000CJA,2022-12-20,3,4,1,066,066,...,00,000,R,DE73,066000000000,06600000,2015-01-01,DEBKGDL20000E6X5,"MULTIPOLYGON (((534242.001 5721821.750, 534214...",66110000
233,81110000,630305.0,2020,DEBKGVG200000CJB,2022-12-20,3,4,1,081,081,...,00,000,R,DE11,081000000000,08100000,1997-01-01,DEBKGDL20000E5Z4,"MULTIPOLYGON (((591667.133 5373246.675, 591782...",81110000
259,82120000,308436.0,2020,DEBKGVG200000CJC,2022-12-20,3,4,1,082,082,...,00,000,R,DE12,082000000000,08200000,1983-06-21,DEBKGDL20000E3WH,"MULTIPOLYGON (((477035.141 5496309.300, 476890...",82120000


In [65]:
auswahl =  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
data_bl = data[data["Kennziffer"].isin(auswahl)]
data_bl = data_bl[data_bl["Zeitbezug"] == 2020]

min_bl = data_bl["Wert"].min()
max_bl = data_bl["Wert"].max()
mean_bl = data_bl["Wert"].mean()
median_bl =data_bl["Wert"].median()

print('Min: ' + str(min_bl) + ', ' + 'Max: ' + str(max_bl) + ', ' + 'Mean: ' + str(mean_bl) + ', ' + 'Median: ' + str(median_bl))

data_bl

Min: 680130.0, Max: 17925570.0, Mean: 5197189.4375, Median: 3287481.5


,Kennziffer,Wert,Zeitbezug
51,1,2910875.0,2020
77,2,1852478.0,2020
103,3,8003421.0,2020
129,4,680130.0,2020
155,5,17925570.0,2020
181,6,6293154.0,2020
207,7,4098391.0,2020
233,8,11103043.0,2020
259,9,13140183.0,2020
285,10,983991.0,2020


In [66]:
data_b = data[data["Kennziffer"] == 111]
data_b = data_b[data_b["Zeitbezug"] == 2020]
data_b

,Kennziffer,Wert,Zeitbezug
25,111,83155031.0,2020
